In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Quantum_Kernel import *
import time

%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit , transpile, Aer ,IBMQ
from qiskit.visualization import *
from qiskit.tools.jupyter import *
from qiskit.providers.ibmq import least_busy
from qiskit.providers.aer import QasmSimulator

from qiskit import Aer, QuantumCircuit
from qiskit.opflow import Z, I, StateFn
from qiskit.utils import QuantumInstance
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit.algorithms.optimizers import *

from qiskit.opflow import StateFn, PauliSumOp, AerPauliExpectation, ListOp, Gradient
from qiskit_machine_learning.neural_networks import OpflowQNN
import qiskit.quantum_info as qi
from scipy import optimize
from qiskit.opflow.gradients import Gradient
from typing import Union

from IPython.display import clear_output
from scipy.optimize import approx_fprime
from tqdm import tqdm

In [3]:
# Loading your IBM Quantum account(s)
from qiskit import IBMQ

# IBMQ.save_account(TOKEN)
IBMQ.load_account() # Load account from disk
provider = IBMQ.get_provider(hub='ibm-q-kqc', group='internal', project='default')

RequestsApiError: "HTTPSConnectionPool(host='auth.quantum-computing.ibm.com', port=443): Max retries exceeded with url: /api/version (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000018B161C3BB0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))"

In [ ]:
def simple_kernel_A(x,repeat=5):
    qc = QuantumCircuit(repeat)
    for i in range(repeat) :
        qc.rx(x[0]*3**i,[i])
    for i in range(repeat-1) :
        qc.cx([i],[i+1])
    qc.cx([repeat-1],[0])
    for i in range(repeat-2) :
        qc.cx([i],[i+2])
    encode = qc
    return [encode,repeat]

In [ ]:
n_samples = 100
blobs_params = dict(random_state=0, n_samples=n_samples, n_features=2)

import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_circles,make_blobs,make_moons,make_classification

np.random.seed(0)

data0_X,data0_y = make_circles(n_samples=n_samples, factor=.5, noise=.025)
data0_X = data0_X*5
data1_X,data1_y = make_moons(n_samples=n_samples, noise=0.025, random_state=0)
data2_X,data2_y = make_blobs(centers=[[2, 2], [-2, -2]], cluster_std=[1, 1], **blobs_params)
data3_X,data3_y = make_blobs(centers=[[1, 1], [-1, -1]], cluster_std=[1, 1], **blobs_params)
data4_X,data4_y = make_classification(n_samples=n_samples,n_features=2, n_redundant=0, n_informative=2, random_state=1, n_clusters_per_class=1)
rng = np.random.RandomState(2)
data4_X += rng.uniform(size=data4_X.shape)*2

datasets_X = [data0_X,data1_X,data2_X,data3_X,data4_X]
datasets_y = [data0_y,data1_y,data2_y,data3_y,data4_y]


In [ ]:
datasets_X_S = []
for i in range(5):
    data_temp = datasets_X[i]
    data_temp_scale = np.pi*(data_temp-np.min(data_temp))/(np.max(data_temp)-np.min(data_temp))
    datasets_X_S += [np.array(data_temp_scale)]

In [ ]:
plt.figure(figsize=(15,15))
for i in range(5):
    reds = datasets_y[i] == 0
    blues = datasets_y[i] == 1

    plt.subplot(5,1,i+1)
    plt.scatter(datasets_X_S[i][reds,0], datasets_X_S[i][reds,1], c="red",s=20, edgecolor='k')
    plt.scatter(datasets_X_S[i][blues,0],datasets_X_S[i][blues,1], c="blue",s=20, edgecolor='k')
    

In [ ]:
def exponential_kernel_B(x,repeat=5):
    qc = QuantumCircuit(repeat)
    for i in range(repeat) :
        qc.rx(x[0]*3**i,[i])
    for i in range(repeat-1) :
        qc.cx([i],[i+1])
    qc.cx([repeat-1],[0])
    for i in range(repeat) :
        qc.ry(x[1]*3**i,[i])
    for i in range(repeat-1) :
        qc.cx([i],[i+1])
    qc.cx([repeat-1],[0])

    encode = qc
    return [encode,repeat]

In [ ]:
datasets_y_sto = []
datasets_test_y_sto =[]
sto_index_sets = []
sto_index_test_sets = []
gram_kernel_set= []
gram_kernel_test_set =[]
gram_gaussian_set = []
gram_gaussian_test_set =[]


sample = 80
test_sample = 20
backend = QasmSimulator()

for i in range(5) :
    sto_ind = np.random.randint(0,len(datasets_X_S[i]),sample)
    data_sto = pd.DataFrame(datasets_X_S[i]).iloc[sto_ind,:]
    data_sto_y =pd.DataFrame(datasets_y[i]).iloc[sto_ind,:]
    sto_index_sets += [sto_ind]
    
    sto_ind = np.random.randint(0,len(datasets_X_S[i]),test_sample)
    data_sto_test = pd.DataFrame(datasets_X_S[i]).iloc[sto_ind,:]
    data_sto_test_y =pd.DataFrame(datasets_y[i]).iloc[sto_ind,:]
    sto_index_test_sets += [sto_ind]
    n = sample
    m = test_sample


    start_time = time.time()
    gram_matrix = get_gram(data=data_sto,kernel_fun=exponential_kernel_B,layer=3,backend=backend,shots=1000)
    gram_test_matrix = get_gram_test(data=data_sto,test_data = data_sto_test,kernel_fun=exponential_kernel_B,layer=3,backend=backend,shots=1000)
    print("--- %s seconds ---" % (time.time() - start_time))
    gram_kernel_set += [gram_matrix]
    gram_kernel_test_set += [gram_test_matrix]

    gram_matrix = Gram_gaussian(data_sto.values,3)
    gram_gaussian_set += [gram_matrix]

    datasets_y_sto += [np.array(data_sto_y)]
    datasets_test_y_sto += [np.array(data_sto_test_y)]

In [ ]:
#for i in range(5):
#    pd.DataFrame(gram_gaussian_set[i]).to_csv("svm_data/gram_gaussian_"+str(i)+".csv")
#    pd.DataFrame(gram_kernel_set[i]).to_csv("svm_data/gram_kernel"+str(i)+".csv")
#    pd.DataFrame(gram_kernel_test_set[i]).to_csv("svm_data/gram_kernel_test"+str(i)+".csv")
#    pd.DataFrame(datasets_X_S[i]).to_csv("svm_data/datasets_X_S"+str(i)+".csv")
#    pd.DataFrame(datasets_y_sto[i]).to_csv("svm_data/datasets_y_sto"+str(i)+".csv")
#    pd.DataFrame(datasets_test_y_sto[i]).to_csv("svm_data/datasets_test_y_sto"+str(i)+".csv")
#    pd.DataFrame(sto_index_sets[i]).to_csv("svm_data/sto_index_sets"+str(i)+".csv")
#    pd.DataFrame(sto_index_test_sets[i]).to_csv("svm_data/sto_index_test_sets"+str(i)+".csv")
    

In [ ]:
y_pred_train = []
y_pred_test = []


for i in range(5):
    y_s = 2*(datasets_y_sto[i]-0.5).T[0]
    diag_y = np.diag(y_s)

    y_s_t= 2*(datasets_test_y_sto[i].T-0.5)[0]
    diag_y_t = np.diag(y_s_t)

    kernel_matrix = gram_kernel_set[i]
    kernel_test_matrix = gram_kernel_test_set[i]

    c_init = np.random.uniform(0,1/upbound,gram_matrix.shape[0])
    c_temp = c_init
    lr = 0.0001
    iter= 30000
    lamda = 1
    upbound = 2*n*lamda
    obj_fun = [0 for k in range(iter)]
    log_bar = [0 for k in range(iter)]

    yky = np.matmul(np.matmul(diag_y,kernel_matrix),diag_y)
    for j in range(iter):
        P = 0.001
        c_n = -sum(c_temp[:(n-1)]*y_s[:(n-1)])*y_s[n-1]
        c_temp = np.array(c_temp[:(n-1)].tolist()+[c_n])
        c_temp += -lr*(np.matmul(yky,c_temp)-1-P*(sum(1/c_temp)+sum(upbound/(1-upbound*c_temp)))/n)
        obj_fun[j] = sum(c_temp)-np.matmul(np.matmul(c_temp.T,yky),c_temp)/2
        log_bar[j] = P*sum(np.log(c_temp)+np.log(upbound**(-1)-c_temp))
    
    plt.subplot(3,5,i+1)
    plt.plot([k for k in range(iter)],obj_fun+log_bar)
    plt.subplot(3,5,i+6)
    plt.plot([k for k in range(iter)],obj_fun)
    plt.subplot(3,5,i+11)
    plt.plot([k for k in range(iter)],log_bar)
        
    b = min(abs(np.matmul((c_temp * y_s),kernel_matrix) - y_s))
    y_pred_train += [np.sign(np.matmul((c_temp * y_s),kernel_matrix) - b)]
    y_pred_test += [np.sign(np.matmul((c_temp * y_s),kernel_test_matrix) - b)]
plt.show()


In [ ]:
[sum(y_pred_train[i]==2*(datasets_y_sto[i]-0.5).T[0])/n for i in range(5)]

In [ ]:
[sum(y_pred_test[i]==2*(datasets_test_y_sto[i]-0.5).T[0])/m for i in range(5)]

In [4]:
def zz_kernel(x,repeat=1):
    p = len(x)
    qc = ZZFeatureMap(p, reps=repeat)
    qc = qc.bind_parameters({qc.parameters[i]:x[i] for i in range(p)})
    encode = qc
    return [encode,p]

In [16]:
def Gram_gaussian(data,comp,version="H") :
    n = data.shape[0];p = data.shape[1] 
    U = np.matmul(data,data.T)
    M = np.outer(np.diag(U),np.ones(shape=(n,1)))
    J = np.outer(np.ones(shape=(n,1)),np.ones(shape=(n,1)))
    Q = np.identity(n)-J/n
    K = M+M.T-2*U
    sigma = (np.sum(np.sqrt(K))-np.trace(np.sqrt(K)))/n/(n-1)
    gamma = comp/sigma/sigma
    result = np.exp(-K*gamma)
    return(result)

def Gram_gaussian_test(data1,data2,comp,version="H") :
    n = data1.shape[0];p = data1.shape[1] 
    m = data2.shape[0]

    U = np.matmul(data1,data1.T)
    M = np.outer(np.diag(U),np.ones(shape=(n,1)))
    K = M+M.T-2*U

    M_new = np.outer(np.diag(U),np.ones(shape=(m,1)))
    U_new = np.matmul(data1,data2.T)
    T_new = np.outer(np.diag(np.matmul(data2,data2.T)),np.ones(shape=(n,1)))
    K_new = M_new+T_new.T-2*U_new

    sigma = (np.sum(np.sqrt(K))-np.trace(np.sqrt(K)))/n/(n-1)
    gamma = comp/sigma/sigma
    result = np.exp(-K_new*gamma)
    return(result)

In [7]:
sto_index_sets = []
sto_index_test_sets =[]
datasets_X_S=[]
datasets_y=[]
for i in range(5):
    sto_index_sets += [pd.read_csv("svm_data/sto_index_sets"+str(i)+".csv").iloc[:,1].tolist()]
    sto_index_test_sets += [pd.read_csv("svm_data/sto_index_test_sets"+str(i)+".csv").iloc[:,1].tolist()]
    datasets_X_S += [pd.read_csv("svm_data/datasets_X_S"+str(i)+".csv").iloc[:,].values.tolist()]
    datasets_y += [pd.read_csv("svm_data/datasets_y"+str(i)+".csv").iloc[:,].values.tolist()]

In [22]:
gram_zz_kernel_set= []
gram_zz_kernel_test_set =[]
gram_gaussian_set = []
gram_gaussian_test_set =[]


sample = 80
test_sample = 20
backend = QasmSimulator()

for i in range(1,5) :
    n = sample
    m = test_sample

    sto_ind = sto_index_sets[i]
    data_sto = pd.DataFrame(datasets_X_S[i]).iloc[sto_ind,:]
    data_sto_y =pd.DataFrame(datasets_y[i]).iloc[sto_ind,:]
    
    sto_ind = sto_index_test_sets[i]
    data_sto_test = pd.DataFrame(datasets_X_S[i]).iloc[sto_ind,:]
    data_sto_test_y =pd.DataFrame(datasets_y[i]).iloc[sto_ind,:]

    gram_matrix = Gram_gaussian(data_sto.values,3)
    gram_matrix_test = Gram_gaussian_test(data_sto.values,data_sto_test.values,3)
    gram_gaussian_set += [gram_matrix]
    gram_gaussian_test_set += [gram_matrix_test]
    start_time = time.time()
    gram_zz_matrix = get_gram(data=data_sto,kernel_fun=zz_kernel,layer=2,backend=backend,shots=1000)
    gram_zz_test_matrix = get_gram_test(data=data_sto,test_data = data_sto_test,kernel_fun=zz_kernel,layer=2,backend=backend,shots=1000)
    
    print("--- %s seconds ---" % (time.time() - start_time))
    gram_zz_kernel_set += [gram_zz_matrix]
    gram_zz_kernel_test_set += [gram_zz_test_matrix]
    
    pd.DataFrame(gram_gaussian_set[i]).to_csv("svm_data/gram_gaussian_"+str(i)+".csv")
    pd.DataFrame(gram_gaussian_test_set[i]).to_csv("svm_data/gram_gaussian_"+str(i)+".csv")
    pd.DataFrame(gram_zz_kernel_set[i]).to_csv("svm_data/gram_kernel"+str(i)+".csv")
    pd.DataFrame(gram_zz_kernel_test_set[i]).to_csv("svm_data/gram_kernel_test"+str(i)+".csv")

 29%|██▉       | 29/100 [2:27:47<6:18:15, 319.66s/it] 

In [ ]:
for i in range(5):
    pd.DataFrame(gram_gaussian_set[i]).to_csv("svm_data/gram_gaussian_"+str(i)+".csv")
    pd.DataFrame(gram_gaussian_test_set[i]).to_csv("svm_data/gram_gaussian_"+str(i)+".csv")
    pd.DataFrame(gram_zz_kernel_set[i]).to_csv("svm_data/gram_kernel"+str(i)+".csv")
    pd.DataFrame(gram_zz_kernel_test_set[i]).to_csv("svm_data/gram_kernel_test"+str(i)+".csv")

In [ ]:
y_pred_train = []
y_pred_test = []

for i in range(5):
    y_s = 2*(datasets_y_sto[i]-0.5).T[0]
    diag_y = np.diag(y_s)

    y_s_t = 2*(datasets_test_y_sto[i].T-0.5)[0]
    diag_y_t = np.diag(y_s_t)

    kernel_matrix = gram_gaussian_set[i]
    kernel_test_matrix = gram_gaussian_test_set[i]

    c_init = np.random.uniform(0,1/upbound,gram_matrix.shape[0])
    c_temp = c_init
    lr = 0.0001
    iter = 30000
    lamda = 1
    upbound = 2*n*lamda
    obj_fun = [0 for k in range(iter)]

    yky = np.matmul(np.matmul(diag_y,kernel_matrix),diag_y)
    
    for j in range(iter):
        P = 0.001
        # Projected Gradient Descent
        c_n = -sum(c_temp[:(n-1)]*y_s[:(n-1)])*y_s[n-1]
        c_temp = np.array(c_temp[:(n-1)].tolist()+[c_n])
        
        # Log- Barrier Methods
        c_temp += -lr*(np.matmul(yky,c_temp)-1-P*(sum(1/c_temp)+sum(upbound/(1-upbound*c_temp)))/n)
        obj_fun[j] = sum(c_temp)-np.matmul(np.matmul(c_temp.T,yky),c_temp)/2
        obj_fun[j] = sum(c_temp)-np.matmul(np.matmul(c_temp.T,yky),c_temp)/2
        log_bar[j] = P*sum(np.log(c_temp)+np.log(upbound**(-1)-c_temp))
    
    plt.subplot(3,5,i+1)
    plt.plot([k for k in range(iter)],obj_fun+log_bar)
    plt.subplot(3,5,i+6)
    plt.plot([k for k in range(iter)],obj_fun)
    plt.subplot(3,5,i+11)
    plt.plot([k for k in range(iter)],log_bar)

    b = np.matmul((c_temp * y_s),kernel_matrix) - y_s
    b_i = abs(b)==min(abs(np.matmul((c_temp * y_s),kernel_matrix) - y_s))
    b = min(abs(b))*np.sign(b[b_i])[0]

    y_pred_train += [np.sign(np.matmul((c_temp * y_s),kernel_matrix) - b)]
    y_pred_test += [np.sign(np.matmul((c_temp * y_s),kernel_test_matrix) - b)]
plt.show()

In [ ]:
[sum(y_pred_train[i]==2*(datasets_y_sto[i]-0.5).T[0])/n for i in range(5)]

In [ ]:
[sum(y_pred_test[i]==2*(datasets_test_y_sto[i]-0.5).T[0])/m for i in range(5)]

In [ ]:
exponential_kernel_B([1,2],3)